In [ ]:
import wikipedia

question = "Who is the current president of America?"

search_results = wikipedia.search(question)
if not search_results:
    print("No relevant Wikipedia content found.")
else:
    try:
        page_title = search_results[0]
        page = wikipedia.page(page_title, auto_suggest=False)  # Avoid auto-suggestions to reduce ambiguity
        content = page.content

    except wikipedia.exceptions.DisambiguationError as e:
        print("Multiple possible pages found:", e.options[:5])
    except wikipedia.exceptions.PageError:
        print("The Wikipedia page could not be found.")


In [ ]:
content

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_text(content)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

local_embeddings = OllamaEmbeddings(model="all-minilm")
vectorstore = Chroma.from_texts(texts=all_splits, embedding=local_embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_chunks = retriever.invoke(question)

In [ ]:
context = ' '.join([doc.page_content for doc in retrieved_chunks])

In [ ]:
context

In [ ]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.2:1b")
response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")

In [ ]:
response